# Customer churn analysis for a bank.
We are using a kaggle dataset where there are some customers with a bank and there are 17 features mapped to it.

Key/Dependent feature here is the exited which means whether the customer has stayed with the bank or not.

So we need to predict whether in the future, customer leaves the bank or not.
 

In [11]:
pip install keras

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#importing the dataset
dataset = pd.read_csv('/content/drive/My Drive/Kaggle Data Sets/Churn_Modelling.csv')
X = dataset.iloc[:,3:13] #Choosing the independent features like credit score, age annual income etc
y = dataset.iloc[:, 13] #Taking out the result ie exited
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


Lets create some dummy variables like gender and geography

In [0]:
geography = pd.get_dummies(X['Geography'], drop_first=True) #Selecting the geography data from X
gender = pd.get_dummies(X['Gender'], drop_first=True)

#Concatenate this with the X
X=pd.concat([X,geography,gender], axis=1)

#dropping the actual columns of geography and gender as they have been added as dummy variables
X=X.drop(['Geography', 'Gender'], axis=1)

Now lets split the data into train test datasets.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # only 20% data is taken for test

Lets scale the inputs to a feasible value. 

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [5]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

Lets define the ANN model now

In [0]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, PReLU, ELU, Dropout

In [16]:
model = Sequential()
#Hidden Layer 1
model.add(Dense(units=6, kernel_initializer= 'he_uniform', activation='relu', input_dim= 11)) #adding first hidden layer with 6 neurons and relu activation
#Hidden Layer 2
model.add(Dense(units=6, kernel_initializer='he_uniform', activation='relu'))#kernel initializer is nothing but the weight initalization method
#Output Layer
model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid')) #Since its binary classificaiton we use sigmoid

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


Lets add a optimzer and Loss function. We will use adam optimizer and since its binary classification we use binary_corssentropy.

In [0]:
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])

Lets train the model

In [20]:
model_history = model.fit(X_train, y_train, validation_split=0.33, batch_size=10, epochs=100)

Epoch 1/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4563 - accuracy: 0.8018 - val_loss: 0.4618 - val_accuracy: 0.8005
Epoch 2/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4421 - accuracy: 0.8056 - val_loss: 0.4494 - val_accuracy: 0.8020
Epoch 3/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4249 - accuracy: 0.8082 - val_loss: 0.4321 - val_accuracy: 0.8039
Epoch 4/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4045 - accuracy: 0.8110 - val_loss: 0.4128 - val_accuracy: 0.8076
Epoch 5/100
536/536 [==============================] - 1s 2ms/step - loss: 0.3882 - accuracy: 0.8140 - val_loss: 0.3995 - val_accuracy: 0.8118
Epoch 6/100
536/536 [==============================] - 1s 2ms/step - loss: 0.3771 - accuracy: 0.8216 - val_loss: 0.3918 - val_accuracy: 0.8160
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.3696 - accuracy: 0.8266 - val_loss: 0.3862 - val_accuracy: 0.8198

In [0]:
pred = model.predict(X_test)
pred= (pred > 0.5)

In [25]:
#Making confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, pred)
cm

array([[1532,   63],
       [ 208,  197]])

In [27]:
score = accuracy_score(pred, y_test)
score

0.8645